In [578]:
import numpy as np
import pandas as pd
import pyodbc
import matplotlib.pyplot as plt

In [579]:

# Solvers 
from scipy.optimize import root as root
from scipy.optimize import brentq as brentq

Parametres

In [580]:
session_date = '2020-07-31'
# If no session id, then take published session
session_id   = None

risk_ids = "('IND_CDP_CAISSE', 'FUND_IND_MSCI_ACWI_EX_CANADA', 'IND_SP_TSX_CAP', 'FUND_IND_DEX_UNIV')"

# Indice CAISSE: IND_CDP_CAISSE
indice_caisse_id = 'IND_CDP_CAISSE';

# Indices de base pour MB     
indice_pour_mb_1_id = 'FUND_IND_MSCI_ACWI_EX_CANADA';
indice_pour_mb_1_percent = 0.80;
indice_pour_mb_2_id = 'IND_SP_TSX_CAP';
indice_pour_mb_2_percent = 0.20;

# Indices de base pour RF      
indice_pour_rf_1_id = 'FUND_IND_DEX_UNIV';
indice_pour_mb_rf_percent = 1.0;

# Calcul de la VaR
percent = 95
metho = 'RobustHD'
timeFactor = 13

In [ ]:
class CalculateVaR:
    # Default values
    percent = 95
    metho = 'RobustHD'
    timeFactor = 13

    def _init_(self, percent=None, metho=None, timeFactor=None)
        if !(percent is None):
            self.percent = percent
        if !(metho is None):
            self.metho = metho
        if !(timeFactor is None):
            self.timeFactor = timeFactor

    def calculate(nb_observation, positionAmount, v_hd, v_pnl):
        # 1- Sort Pnls
        #print(v_pnl)
        v_pnl_sorted = np.sort(v_pnl)
        #print(v_pnl_sorted)

        # 3 sort HD 
        v_hdWeights_sorted = v_hdWeights.sort_values('Rang')['Poids']

        # Multiply Pnl by HD
        print(v_pnl_sorted.shape, v_hdWeights_sorted.shape) 
        v_pnl_hd = v_pnl_sorted * v_hdWeights_sorted
        # print(v_pnl_hd.shape)

        # Average, Percentile
        average = np.average(v_pnl)
        sum = np.sum(v_pnl_hd)
        percentile = sum 
        # print(average, sum, percentile)

        # Annualized
        annualizedVaR = 1- pow(1 + (percentile - average) / positionAmount, sqrt(timeFactor))
        # return var 
        return annualizedVaR


In [581]:
print(pyodbc)
con = pyodbc.connect('Driver={SQL Server};'
                      'Server=CECGENSQL4\MECZAPRSK01;'
                      'Database=rsq_model;'
                      'Trusted_Connection=yes;')
print(con)

<module 'pyodbc' from 'C:\\ProgramData\\Anaconda3\\envs\\DLENV\\lib\\site-packages\\pyodbc.cp37-win_amd64.pyd'>


conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=server_name;'
                      'Database=db_name;'
                      'Trusted_Connection=yes;')


In [582]:
query_template = """
SELECT   
	  IndiceIDRisque
	  , [SessionDate]
      ,[SessionID]
      ,[DynamicIdentifier]
      ,[CaracteristiqueScenarioLibelle]
      ,[IndicateurDonneePubliee]
      ,[FrequenceSessionCode]
      ,[TypeScenario]
      ,[HorizonCalcul]
      ,[NombreObservation]
      ,[ValeurMarchande]
      , A.Date as ScenarioDate
	  , A.Pnl  as Pnl
	  , A.Pnl/ValeurMarchande as Pnl_percent
  FROM  [MesureRisqueMarche].[V_PnLRMIndiceVecteur] V
  cross apply [dbo].[PivotDatePnl] ([VecteurScenariodate], vecteurIndicePnl, ';') A
  where SessionDate = '{}'
  and {}
  and IndiceIDRisque in {}
  
"""

if session_id is None:
    query_session_selection = 'IndicateurDonneePubliee = 1'
else:
    query_session_selection = 'sessionId = {}'.format(session_id)

query = query_template.format(session_date, query_session_selection, risk_ids)

print(query)


SELECT   
	  IndiceIDRisque
	  , [SessionDate]
      ,[SessionID]
      ,[DynamicIdentifier]
      ,[CaracteristiqueScenarioLibelle]
      ,[IndicateurDonneePubliee]
      ,[FrequenceSessionCode]
      ,[TypeScenario]
      ,[HorizonCalcul]
      ,[NombreObservation]
      ,[ValeurMarchande]
      , A.Date as ScenarioDate
	  , A.Pnl  as Pnl
	  , A.Pnl/ValeurMarchande as Pnl_percent
  FROM  [MesureRisqueMarche].[V_PnLRMIndiceVecteur] V
  cross apply [dbo].[PivotDatePnl] ([VecteurScenariodate], vecteurIndicePnl, ';') A
  where SessionDate = '2020-07-31'
  and IndicateurDonneePubliee = 1
  and IndiceIDRisque in ('IND_CDP_CAISSE', 'FUND_IND_MSCI_ACWI_EX_CANADA', 'IND_SP_TSX_CAP', 'FUND_IND_DEX_UNIV')
  



In [583]:
# Load data
data = pd.read_sql(query, con)

In [584]:
data.shape

(15136, 14)

In [585]:
data.NombreObservation.unique()[0]

3784

In [586]:
data.head(10)

,IndiceIDRisque,SessionDate,SessionID,DynamicIdentifier,CaracteristiqueScenarioLibelle,IndicateurDonneePubliee,FrequenceSessionCode,TypeScenario,HorizonCalcul,NombreObservation,ValeurMarchande,ScenarioDate,Pnl,Pnl_percent
0,FUND_IND_DEX_UNIV,2020-07-31,2616,CDPQ_20200731_Daily_2,Historique_4W_20060131,True,D,historique,20,3784,1.337651e+11,2020-07-31,1.363377e+09,0.010192
1,FUND_IND_DEX_UNIV,2020-07-31,2616,CDPQ_20200731_Daily_2,Historique_4W_20060131,True,D,historique,20,3784,1.337651e+11,2020-07-30,1.388943e+09,0.010383
2,FUND_IND_DEX_UNIV,2020-07-31,2616,CDPQ_20200731_Daily_2,Historique_4W_20060131,True,D,historique,20,3784,1.337651e+11,2020-07-29,1.091948e+09,0.008163
3,FUND_IND_DEX_UNIV,2020-07-31,2616,CDPQ_20200731_Daily_2,Historique_4W_20060131,True,D,historique,20,3784,1.337651e+11,2020-07-28,1.172368e+09,0.008764
4,FUND_IND_DEX_UNIV,2020-07-31,2616,CDPQ_20200731_Daily_2,Historique_4W_20060131,True,D,historique,20,3784,1.337651e+11,2020-07-27,8.736499e+08,0.006531
5,FUND_IND_DEX_UNIV,2020-07-31,2616,CDPQ_20200731_Daily_2,Historique_4W_20060131,True,D,historique,20,3784,1.337651e+11,2020-07-24,1.278547e+09,0.009558
6,FUND_IND_DEX_UNIV,2020-07-31,2616,CDPQ_20200731_Daily_2,Historique_4W_20060131,True,D,historique,20,3784,1.337651e+11,2020-07-23,1.160141e+09,0.008673
7,FUND_IND_DEX_UNIV,2020-07-31,2616,CDPQ_20200731_Daily_2,Historique_4W_20060131,True,D,historique,20,3784,1.337651e+11,2020-07-22,1.337441e+09,0.009998
8,FUND_IND_DEX_UNIV,2020-07-31,2616,CDPQ_20200731_Daily_2,Historique_4W_20060131,True,D,historique,20,3784,1.337651e+11,2020-07-21,1.195991e+09,0.008941
9,FUND_IND_DEX_UNIV,2020-07-31,2616,CDPQ_20200731_Daily_2,Historique_4W_20060131,True,D,historique,20,3784,1.337651e+11,2020-07-20,8.228439e+08,0.006151


In [587]:
data.IndiceIDRisque

0        FUND_IND_DEX_UNIV
1        FUND_IND_DEX_UNIV
2        FUND_IND_DEX_UNIV
3        FUND_IND_DEX_UNIV
4        FUND_IND_DEX_UNIV
               ...        
15131       IND_SP_TSX_CAP
15132       IND_SP_TSX_CAP
15133       IND_SP_TSX_CAP
15134       IND_SP_TSX_CAP
15135       IND_SP_TSX_CAP
Name: IndiceIDRisque, Length: 15136, dtype: object

In [588]:
is_indice_caisse = data.IndiceIDRisque == indice_caisse_id
data[is_indice_caisse] 

,IndiceIDRisque,SessionDate,SessionID,DynamicIdentifier,CaracteristiqueScenarioLibelle,IndicateurDonneePubliee,FrequenceSessionCode,TypeScenario,HorizonCalcul,NombreObservation,ValeurMarchande,ScenarioDate,Pnl,Pnl_percent
7568,IND_CDP_CAISSE,2020-07-31,2616,CDPQ_20200731_Daily_2,Historique_4W_20060131,True,D,historique,20,3784,9.999996e+10,2020-07-31,2.442538e+09,0.024425
7569,IND_CDP_CAISSE,2020-07-31,2616,CDPQ_20200731_Daily_2,Historique_4W_20060131,True,D,historique,20,3784,9.999996e+10,2020-07-30,2.608825e+09,0.026088
7570,IND_CDP_CAISSE,2020-07-31,2616,CDPQ_20200731_Daily_2,Historique_4W_20060131,True,D,historique,20,3784,9.999996e+10,2020-07-29,3.322061e+09,0.033221
7571,IND_CDP_CAISSE,2020-07-31,2616,CDPQ_20200731_Daily_2,Historique_4W_20060131,True,D,historique,20,3784,9.999996e+10,2020-07-28,2.916263e+09,0.029163
7572,IND_CDP_CAISSE,2020-07-31,2616,CDPQ_20200731_Daily_2,Historique_4W_20060131,True,D,historique,20,3784,9.999996e+10,2020-07-27,3.225027e+09,0.032250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11347,IND_CDP_CAISSE,2020-07-31,2616,CDPQ_20200731_Daily_2,Historique_4W_20060131,True,D,historique,20,3784,9.999996e+10,2006-02-06,-1.129640e+08,-0.001130
11348,IND_CDP_CAISSE,2020-07-31,2616,CDPQ_20200731_Daily_2,Historique_4W_20060131,True,D,historique,20,3784,9.999996e+10,2006-02-03,-5.188118e+07,-0.000519
11349,IND_CDP_CAISSE,2020-07-31,2616,CDPQ_20200731_Daily_2,Historique_4W_20060131,True,D,historique,20,3784,9.999996e+10,2006-02-02,7.246380e+08,0.007246
11350,IND_CDP_CAISSE,2020-07-31,2616,CDPQ_20200731_Daily_2,Historique_4W_20060131,True,D,historique,20,3784,9.999996e+10,2006-02-01,1.369914e+09,0.013699


In [589]:
v_caisse = data[data.IndiceIDRisque == indice_caisse_id].Pnl_percent
v_caisse

7568     0.024425
7569     0.026088
7570     0.033221
7571     0.029163
7572     0.032250
           ...   
11347   -0.001130
11348   -0.000519
11349    0.007246
11350    0.013699
11351    0.019798
Name: Pnl_percent, Length: 3784, dtype: float64

In [590]:
v_mb_1 = data[data.IndiceIDRisque == indice_pour_mb_1_id].Pnl_percent
v_mb_1

3784    0.029190
3785    0.031194
3786    0.044019
3787    0.037868
3788    0.046209
          ...   
7563   -0.027625
7564   -0.019153
7565    0.000834
7566    0.021256
7567    0.029752
Name: Pnl_percent, Length: 3784, dtype: float64

In [591]:
v_mb_2 = data[data.IndiceIDRisque == indice_pour_mb_2_id].Pnl_percent
v_mb_2

11352    0.043649
11353    0.049169
11354    0.055499
11355    0.044101
11356    0.055875
           ...   
15131    0.041387
15132    0.030010
15133    0.038350
15134    0.035920
15135    0.035089
Name: Pnl_percent, Length: 3784, dtype: float64

In [592]:
v_rf_1 = data[data.IndiceIDRisque == indice_pour_rf_1_id].Pnl_percent
v_rf_1

0       0.010192
1       0.010383
2       0.008163
3       0.008764
4       0.006531
          ...   
3779   -0.008732
3780   -0.008231
3781   -0.012719
3782   -0.015960
3783   -0.011695
Name: Pnl_percent, Length: 3784, dtype: float64

In [593]:
v_mb = indice_pour_mb_1_percent * np.array(v_mb_1) + indice_pour_mb_2_percent * np.array(v_mb_2)
v_mb

array([0.03208173, 0.034789  , 0.046315  , ..., 0.0083369 , 0.02418846,
       0.03081962])

In [594]:
v_rf = indice_pour_mb_rf_percent * np.array(v_rf_1)  
v_rf

array([ 0.01019232,  0.01038345,  0.00816318, ..., -0.01271892,
       -0.0159602 , -0.01169474])

In [595]:
# Get Poids HD
def getHDWeights(percent, metho, nb_observation):
    hd_query = """
    SELECT 
        -- [Methodologie]
        --,[NombreObservation]
        --,[Percentile],
        [Rang]
        ,[Poids]
    FROM [rsq_model].[MesureRisqueMarche].[V_PonderationHarrellDavis]
    where Methodologie = '{}' and  [Percentile] = {} and 
    NombreObservation =  {} 
    """ 
    hd = pd.read_sql(hd_query.format(metho, percent, nb_observation), con)
    hd.head()

    # Sort by Rang
    return hd.sort_values('Rang')

In [596]:
hd.shape

(3000, 2)

In [597]:
percent = 95
metho = 'RobustHD'
nb_observation = data.NombreObservation.unique()[0]
print(nb_observation)

hd = getHDWeights(percent, metho, nb_observation)
# hd['Poids']

3784


In [598]:
from math import sqrt, pow

def calculVaR(percent, metho, nb_observation, timeFactor, positionAmount ,  v_pnl): 
# Fonction qui calcule la VaR des pnls considérés 
    # 1- Poids HD
    v_hdWeights = getHDWeights(percent, metho, nb_observation)
    #print(hdWeights.head())

    # 2- Sort Pnls
    #print(v_pnl)
    v_pnl_sorted = np.sort(v_pnl)
    #print(v_pnl_sorted)

    # 3 sort HD 
    v_hdWeights_sorted = v_hdWeights.sort_values('Rang')['Poids']

    # Multiply Pnl by HD
    print(v_pnl_sorted.shape, v_hdWeights_sorted.shape) 
    v_pnl_hd = v_pnl_sorted * v_hdWeights_sorted
    # print(v_pnl_hd.shape)

    # Average, Percentile
    average = np.average(v_pnl)
    sum = np.sum(v_pnl_hd)
    percentile = sum 
    # print(average, sum, percentile)

    # Annualized
    annualizedVaR = 1- pow(1 + (percentile - average) / positionAmount, sqrt(timeFactor))
    # return var 
    return annualizedVaR
 

In [599]:
nb_observation = data.NombreObservation.unique()[0]
positionAmount = 1 # Valeur marchande = 1 si %
print(nb_observation)


3784


In [1]:
# VaR Caisse
var_Caisse = calculVaR(percent, metho, nb_observation, timeFactor, positionAmount, v_caisse) 
print(var_Caisse)
var_target = var_Caisse

#print(v_caisse)

NameError: name 'calculVaR' is not defined

In [601]:
var_mb_1 = calculVaR(percent, metho, nb_observation, timeFactor, positionAmount, v_mb_1) 
print(var_mb_1)
 

(3784,) (3784,)
0.22807591472230526


In [602]:
var_mb_2 = calculVaR(percent, metho, nb_observation, timeFactor, positionAmount, v_mb_2) 
print(var_mb_2)
 

(3784,) (3784,)
0.20747176785653287


In [603]:
var_mb = calculVaR(percent, metho, nb_observation, timeFactor, positionAmount, v_mb) 
print(var_mb)
print(v_mb)

(3784,) (3784,)
0.2183916150414441
[0.03208173 0.034789   0.046315   ... 0.0083369  0.02418846 0.03081962]


In [604]:
var_rf_1 = calculVaR(percent, metho, nb_observation, timeFactor, positionAmount, v_rf_1) 
print(var_rf_1)


(3784,) (3784,)
0.06761877371321512


In [605]:
var_rf = calculVaR(percent, metho, nb_observation, timeFactor, positionAmount, v_rf) 
print(var_rf)
print(v_rf)

(3784,) (3784,)
0.06761877371321512
[ 0.01019232  0.01038345  0.00816318 ... -0.01271892 -0.0159602
 -0.01169474]


In [606]:
def getVectorPLS(x:float):
    pnl =  x * v_mb + (1-x) * v_rf
    print(pnl[:10])
    print(pnl.shape)
    return pnl  

In [607]:
def calculateVaRPLS(x:float):
    pnl =  getVectorPLS(x)
    var = calculateVaR(pnl)
    return abs(var)  

In [608]:
# Fonction a optimiser
def toOptimize(x:float):
    var = calculateVaRPLS(x)
    return abs(var) - abs(var_target)

In [609]:
print(var_target)
 

0.13577972791668258


In [610]:
# Optimize using root

sol = root(toOptimize, 1)
sol

[0.03208173 0.034789   0.046315   0.03911474 0.04814181 0.04814381
 0.04275199 0.0643721  0.04531693 0.0523144 ]
(3784,)
[0.03208173 0.034789   0.046315   0.03911474 0.04814181 0.04814381
 0.04275199 0.0643721  0.04531693 0.0523144 ]
(3784,)
[0.03208173 0.034789   0.046315   0.03911474 0.04814181 0.04814381
 0.04275199 0.0643721  0.04531693 0.0523144 ]
(3784,)
[0.03208173 0.034789   0.046315   0.03911474 0.04814182 0.04814381
 0.04275199 0.0643721  0.04531693 0.0523144 ]
(3784,)
[0.02226879 0.02384809 0.02921169 0.0255088  0.02948796 0.03084602
 0.02747451 0.03999661 0.02900974 0.03161972]
(3784,)
[0.02150365 0.022995   0.0278781  0.0244479  0.02803346 0.02949726
 0.02628328 0.03809598 0.02773822 0.0300061 ]
(3784,)
[0.0214594  0.02294565 0.02780097 0.02438654 0.02794933 0.02941925
 0.02621438 0.03798605 0.02766468 0.02991276]
(3784,)


    fjac: array([[-1.]])
     fun: 0.0
 message: 'The solution converged.'
    nfev: 5
     qtf: array([-0.00042067])
       r: array([-0.20806438])
  status: 1
 success: True
       x: array([0.51472737])

Résultat 1: Méthode root

In [611]:
x1 = sol.x[0]
print(x1)

0.5147273736090315


In [612]:
getVectorPLS(x1)

[0.0214594  0.02294565 0.02780097 0.02438654 0.02794933 0.02941925
 0.02621438 0.03798605 0.02766468 0.02991276]
(3784,)


array([ 0.0214594 ,  0.02294565,  0.02780097, ..., -0.00188091,
        0.00470541,  0.01018857])

In [613]:
calculateVaRPLS(x1)

[0.0214594  0.02294565 0.02780097 0.02438654 0.02794933 0.02941925
 0.02621438 0.03798605 0.02766468 0.02991276]
(3784,)


0.13577972791668258

Résultat 2: Méthode brentq

In [614]:
#Optimize using brentq
x2 = brentq(toOptimize, 0, 1)

print(x2)

[0.01019232 0.01038345 0.00816318 0.00876438 0.00653122 0.00955815
 0.00867297 0.00999843 0.00894098 0.00615141]
(3784,)
[0.03208173 0.034789   0.046315   0.03911474 0.04814181 0.04814381
 0.04275199 0.0643721  0.04531693 0.0523144 ]
(3784,)
[0.01973658 0.0210248  0.02479821 0.0219978  0.02467435 0.02638234
 0.02353217 0.03370654 0.02480169 0.02627948]
(3784,)
[0.02179324 0.02331787 0.02838283 0.02484942 0.02858395 0.03000773
 0.02673413 0.03881532 0.02821946 0.03061681]
(3784,)
[0.0214594  0.02294565 0.02780097 0.02438654 0.02794933 0.02941925
 0.02621438 0.03798605 0.02766468 0.02991276]
(3784,)
[0.0214594  0.02294565 0.02780097 0.02438654 0.02794933 0.02941925
 0.02621438 0.03798605 0.02766468 0.02991276]
(3784,)
0.5147273736090316


In [615]:
toOptimize(x2)

[0.0214594  0.02294565 0.02780097 0.02438654 0.02794933 0.02941925
 0.02621438 0.03798605 0.02766468 0.02991276]
(3784,)


2.7755575615628914e-17

In [616]:
y = calculateVaRPLS (x2)
print(y)

[0.0214594  0.02294565 0.02780097 0.02438654 0.02794933 0.02941925
 0.02621438 0.03798605 0.02766468 0.02991276]
(3784,)
0.1357797279166826


In [617]:
getVectorPLS(x2)

[0.0214594  0.02294565 0.02780097 0.02438654 0.02794933 0.02941925
 0.02621438 0.03798605 0.02766468 0.02991276]
(3784,)


array([ 0.0214594 ,  0.02294565,  0.02780097, ..., -0.00188091,
        0.00470541,  0.01018857])